# Example of beta reconstruction usage

This notebook gives a basic example of how to do a beta reconstruction analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from defdap.quat import Quat

from defdap.plotting import MapPlot

from beta_reconstruction.reconstruction import do_reconstruction, load_map, assign_beta_variants

%matplotlib qt

## Load in EBSD file
The `load_map` function will load the EBSD map from the specified file and
do prerequisite calculations such as finding grains and grain boundaries and
constructing a neighbour network of the grain boundaries.

In [ ]:
ebsd_file_path = "example_data/ZrNb_triplepoint.ctf"

EbsdMap = load_map(ebsd_file_path)

## Plot EBSD map
We can plot the alpha grain structure before reconstruction.

In [ ]:
EbsdMap.plotEulerMap()

## Calculate possible beta orientations 

There is a relationship between the $\alpha$ and $\beta$ symmetries, such that for each alpha orientation there are 6 theoretically possible $\beta$ orientations. This called the Burgers relationship.

The set of 6 possible orientations can be narrowed down further (often to a unique solution) by also considering the orientations of neighbouring $\alpha$ grains which are inherited from a single $\beta$ grain. Being inheried from the same $\beta$ grain restrict the possible neighbouring misorientation types between different $\alpha$ grains. In reverse, this means that certain combinations of misorientations can be used to determine the prior $\beta$ orientation.

In [ ]:
do_reconstruction(EbsdMap, burg_tol=5., ori_tol=3.)

The results are stored in the grain objects which comprise the EBSD map.

If running with interactive plots the `locateGrainID()` function should plot an interactive EBSD map. Clicking on a grain will print the grain ID to the notebook cell. This grain ID can then be used to get information about the grain by subsetting the Map object with the grain ID.

In [ ]:
EbsdMap.locateGrainID()

The reconstruction algorithm first calculates the 6 possible $\beta$ orientations for the grain given its $\alpha$ orientation. These are are stored in the `betaOris` variable.

In [ ]:
grain = EbsdMap[8]
grain.betaOris

The next step involves calculating the misorientation between the grain and its neighbours. This is stored in the `betaDeviations` variable. Neighbiouring grains are only conisdered if the misorientation between them and the grain is lower than the misorientation threshold set in the call to the `do_reconstruction()` method.

In [ ]:
np.array(grain.betaDeviations) *180 /np.pi

The `possibleBetaOris` variable stores the possible beta orientations of the parent $\beta$ grain as a result of considering the misorientation relation between the neighbouring $\alpha$ grains. There is one list of possible orientations for each neighbour.

In [ ]:
grain.possibleBetaOris

The `possibleBetaOris` are then binned into one of the six `betaOris` with a tolerance determined by the `ori_tol` variable passed to the `doReconstruction()` function. This is essentially a vote on which parent $\beta$ grain orientation is most likely. The `variantCount` variable stores the counts for each possible beta orientation.

In [ ]:
grain.variantCount

## Identification of the most likely $\beta$ variant

We can now plot the six possible beta orientations on an ODF map. Those marked with a cross are those which have a non-zero variant count.

**<font color=red>What does this tell us conceptually? Presumably it is relevant that all of the selected variants are proximate?</font>**

In [ ]:
possibleBetaOris = [item for sublist in grain.possibleBetaOris for item in sublist]

directions = [
    np.array([1,0,0]), 
    np.array([0,1,0]), 
    np.array([0,0,1])
]
markerSize = 100

fig, axes = plt.subplots(1, len(directions))

for direction, ax in zip(directions, axes):
    plot = Quat.plotIPF(grain.betaOris, direction, "cubic", marker='o', s=markerSize, fig=fig, ax=ax)
    Quat.plotIPF(possibleBetaOris, direction, "cubic", s=markerSize,  plot=plot)

fig.tight_layout()

## Find the most common variant for each grain and set this as the beta orientaion

In the simplest interpretation of the variant counts we can consider the orientation of the parent $\beta$ grain to be the mode variant. Where there are two variants with the same count we do not assign the orientation.

In [ ]:
EbsdMap = assign_beta_variants(EbsdMap, "modal")

## Plot variant map

Now we have assigned a the mode variant to each grain we can plot this as a map.

In [ ]:
modalVariants = [grain.modeVariant for grain in EbsdMap]

plot = EbsdMap.plotGrainDataMap(grainData=modalVariants, vmin=-1, vmax=5, cmap="Set1")
plot.addColourBar("modal variant")

## IPF of grains filled with average orientation

<font color=red>I don't really understand what is going on in these last few cells - how the chosen beta varient is tranformed into orientations. Some of this probably needs to go into a function in the code file and then add a few words to explain here.</font>

In [ ]:
parentBetaOris = [grain.parentBetaOri for grain in EbsdMap]

directions = [
    np.array([1,0,0]),
    np.array([0,1,0]),
    np.array([0,0,1])
]

fig, axes = plt.subplots(1, len(directions), figsize=(15,5))

for ax, direction in zip(axes, directions):
    betaGrainIPFColours = Quat.calcIPFcolours(parentBetaOris, direction, "cubic")

    EbsdMap.plotGrainDataMap(grainData=betaGrainIPFColours, fig=fig, ax=ax)
    
fig.tight_layout()

## Calculate and then plot beta orientation map

In [ ]:
from beta_reconstruction.crystal_relations import unq_hex_syms, burg_trans

transformations = []
for sym in unq_hex_syms:
    transformations.append(burg_trans * sym.conjugate)
    
variantMap = EbsdMap.grainDataToMapData(modeVariants, bg=-2)
    
betaQuatArray = np.empty_like(EbsdMap.quatArray)
for i in range(EbsdMap.yDim):
    for j in range(EbsdMap.xDim):
        variant = variantMap[i, j]
        if variant < 0:
            # points not part of a grain (-2) and 
            # those that were not reconstructed (-1)
            betaQuatArray[i, j] = Quat(1, 0, 0, 0)
        else:
#             betaQuatArray[i, j] = burg_trans * unq_hex_syms[variant].conjugate * EbsdMap.quatArray[i, j]
            betaQuatArray[i, j] = transformations[variant] * EbsdMap.quatArray[i, j]

In [ ]:
directions = [
    np.array([1,0,0]),
    np.array([0,1,0]),
    np.array([0,0,1])
]

fig, axes = plt.subplots(1, len(directions), figsize=(15,5))

for ax, direction in zip(axes, directions):
    betaIPFColours = Quat.calcIPFcolours(betaQuatArray.flatten(), direction, "cubic")
    # reshape back to map shape array
    betaIPFColours = np.reshape(betaIPFColours, (EbsdMap.yDim, EbsdMap.xDim, 3))
    
    # recolour the -1 and -2 variants
    # -1 grains not reconstructed (white)
    # -2 clusters too small to be a grain (black)
    indxs = np.where(variantMap == -1)
    betaIPFColours[indxs[0], indxs[1]] = np.array([1, 1, 1])
    indxs = np.where(variantMap == -2)
    betaIPFColours[indxs[0], indxs[1]] = np.array([0, 0, 0])
    
    plot = MapPlot.create(EbsdMap, betaIPFColours, fig=fig, ax=ax)
    
fig.tight_layout()